In [ ]:
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F

df = pd.read_csv('https://assets.datacamp.com/production/repositories/6193/datasets/fca5067912db2b2346f568ce806915450fe56b99/water_potability.csv').to_numpy()
zero = df[100, :]

sample = torch.tensor([zero[:9]]).float()
target = torch.tensor([[1, 0]]).float()

weight = torch.randn(2, 9, requires_grad=True) 
bias = torch.randn(2, requires_grad=True)

preds = sample.matmul(weight.t()) + bias
criterion = nn.CrossEntropyLoss()

# Calculate the loss
loss = criterion(preds, target)

# Compute the gradients of the loss
loss.backward()

# Display gradients of the weight and bias tensors in order
print(weight.grad)
print(bias.grad)

In [ ]:
y_hat = np.array(10)
y = np.array(1)

# Calculate the MSELoss using NumPy
mse_numpy = np.mean((y_hat-y)**2)

# Create the MSELoss function
criterion = nn.MSELoss()

# Calculate the MSELoss using the created loss function
mse_pytorch = criterion(torch.tensor(y_hat).float(), torch.tensor(y).float())
print(mse_pytorch)

In [ ]:
# Loop over the number of epochs and the dataloader
for i in range(num_epochs):
  for data in dataloader:
    # Set the gradients to zero
    optimizer.zero_grad()
    # Run a forward pass
    feature, target = data
    prediction = model(feature)    
    # Calculate the loss
    loss = criterion(prediction, target)    
    # Compute the gradients
    loss.backward()
    # Update the model's parameters
    optimizer.step()
show_results(model, dataloader)

In [ ]:
def calculate_capacity(model):
  total = 0
  for p in model.parameters():
    total += p.numel()
  return total

n_features = 8
n_classes = 2

input_tensor = torch.Tensor([[3, 4, 6, 2, 3, 6, 8, 9]])

# Create a neural network with more than 120 parameters
model = nn.Sequential(nn.Linear(n_features, 12), nn.Linear(12,8), nn.Linear(8,4),
nn.Linear(4, n_classes))

output = model(input_tensor)

print(calculate_capacity(model))